In [19]:
import pandas as pd
import openpyxl
import xlrd
import pyarrow as pa
import pyarrow.parquet as pq

import warnings
warnings.filterwarnings(action= 'ignore')

%load_ext autoreload
%autoreload 2
import resources as r




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
# Cargar el archivo Excel
file_path = '../data/raw/info_contable_ypf_2020_1.xls'  # path del fichero.xlsx
workbook = xlrd.open_workbook(file_path) # instancio la carga del fichero en una variable

# Crear un diccionario para almacenar los DataFrames de las tablas dinámicas
pivot_tables = {} # instancio un diccionario para guardar la etiqueta de la hoja y su data en formato dataframe

# Iterar sobre cada hoja del archivo
for idx, sheet_name in enumerate(workbook.sheet_names(), start=1): # itero sobre cada hoja tomando como referencia las etiquetas de cada una con el metodo sheetnames

    sheet = workbook.sheet_by_name(sheet_name)  # Instancia la hoja
    
    # Identificar la tabla dinámica (puedes ajustar este criterio según tu necesidad)
    # Aquí asumimos que la tabla dinámica empieza en la fila 2 y columna B (cambiar según sea necesario)
    pivot_start_row = 1  # Las filas en xlrd están indexadas desde 0
    pivot_start_col = 1  # Las columnas en xlrd están indexadas desde 0
    
    # Extraer la tabla dinámica
    pivot_data = []
    for row_idx in range(pivot_start_row, sheet.nrows):
        row = sheet.row_values(row_idx, start_colx=pivot_start_col)
        pivot_data.append(row)
    
    # Convertir a DataFrame de pandas
    pivot_df = pd.DataFrame(pivot_data[1:], columns = pivot_data[0])
    
    # # Almacenar la tabla dinámica en el diccionario
    pivot_tables[sheet_name] = pivot_df

# Mostrar las tablas dinámicas extraídas
for sheet_name, pivot_table in pivot_tables.items(): # iteramos sobre el nombre de la hoja (clave) y sobre el dataframe con los datos (valor) del diccionario
    año = 2020 # inicializamos una variable que contenga el año al que refieren los datos
    print(f'Tabla dinámica en la hoja: {sheet_name}')
    # print(pivot_table)
    # print('\n')
    df = pd.DataFrame(pivot_tables[sheet_name]) # instanciamos los datos de cada hoja en un dataframe
    df_transposed = df.T # transponemos los datos

    df_transposed.columns = pivot_table.iloc[:, 0] 
    df_transposed = df_transposed.drop(df_transposed.index[0])
    df_transposed['Año'] = año

    if '' in df_transposed.columns:
        df_transposed.drop(columns= '', inplace= True)


    archivo= f'../data/raw/info_contable/{sheet_name}.parquet'
    try:
        pq.write_table(pa.Table.from_pandas(df_transposed), archivo)
    except ValueError:
        df_transposed.columns = list(r.make_unique(df_transposed.columns))
        pq.write_table(pa.Table.from_pandas(df_transposed), archivo)
    
    print(df_transposed.head())
    print(df_transposed.index)
    print('-'*100)
    print()
# print(f'Nombre de dataframe: {sheet_name}')
# pivot_df.head()


Tabla dinámica en la hoja: P&L
        (Amounts expressed in million of Argentine Pesos)   Revenues  \
1Q 2019                                                     130907.0   
4Q 2019                                                     206910.0   
1Q 2020                                                     174670.0   

            Costs Gross Profit Selling Expenses Administration Expenses  \
1Q 2019 -104754.0      26153.0          -9820.0                 -4768.0   
4Q 2019 -187044.0      19866.0         -16963.0                 -8124.0   
1Q 2020 -145914.0      28756.0         -13876.0                 -6749.0   

        Exploration Expenses Other operating results, net Operating Income  \
1Q 2019              -1521.0                        587.0          10631.0   
4Q 2019              -2348.0                       -617.0          -8186.0   
1Q 2020               -716.0                       7383.0          14798.0   

        Income (loss) of interest in companies and joint ventures 

In [13]:
## Extraemos la tabla estatica de inversiones por empresa por año.

# r.conversion_de_tablas_dinamicas('../data/raw/TD Inversiones_2024.xlsx', 3, 12, 'A', 2024)
# r.conversion_de_tablas_dinamicas('../data/raw/TD Inversiones_2023.xlsx', 3, 12, 'A', 2023)
# r.conversion_de_tablas_dinamicas('../data/raw/TD Inversiones_2022.xlsx', 3, 12, 'A', 2022)
# r.conversion_de_tablas_dinamicas('../data/raw/TD Inversiones_2021.xlsx', 3, 14, 'A', 2021)
# r.conversion_de_tablas_dinamicas('../data/raw/td_2057_2020.xlsx', 3, 12, 'A', 2020)

In [27]:
file_path = '../data/raw/reservas al 31-12-2022.xlsx'  # path del fichero.xlsx
workbook = openpyxl.load_workbook(file_path, data_only=True) # instancio la carga del fichero en una variable

# Iterar sobre cada hoja del archivo
for idx, sheet_name in enumerate(workbook.sheetnames, start=1): # itero sobre cada hoja tomando como referencia las etiquetas de cada una con el metodo sheetnames
    
    if idx != 2:  # Solo procesar la segunda hoja
        continue

    sheet = workbook[sheet_name] # instancio la hoja
    
    # Identificar la tabla dinámica (puedes ajustar este criterio según tu necesidad)
    # Aquí asumimos que la tabla dinámica empieza en la fila 12 y columna A (cambiar según sea necesario)
    pivot_start_row = 7
    pivot_start_col = 'A'
    
    # Extraer la tabla dinámica
    pivot_data = []
    for row in sheet.iter_rows(min_row=pivot_start_row, min_col=openpyxl.utils.cell.column_index_from_string(pivot_start_col), values_only=True):
        pivot_data.append(row)

    columnas = list(pivot_data[0])
    new_columns = []
    num = 1
    for idx, columna in enumerate(columnas):
        if idx >= 21:
            if idx == 21:
                temp = 'reserv_comp_pet_' + columna
            elif idx == 22:
                temp = 'reserv_comp_gas_' + columna
            elif idx == 23:
                temp = 'reserv_prob_pet_' + columna
            elif idx == 24:
                temp = 'reserv_prob_gas_' + columna
            elif idx == 25:
                temp = 'reserv_pos_pet_' + columna
            elif idx == 26:
                temp = 'reserv_pos_gas_' + columna
            elif idx == 27:
                temp = 'recur_cont_pet_' + columna
            elif idx == 28:
                temp = 'recur_cont_gas_' + columna
            new_columns.append(temp)
        elif idx > 4 and idx < 21:
            temp = str(num) + columna
            num += 1
            new_columns.append(temp)
        else:
            new_columns.append(columna)

    print(columnas)
    print(len(columnas))
    print(new_columns)
    print(new_columns[5:21])
    
    # Convertir a DataFrame de pandas
    pivot_df = pd.DataFrame(pivot_data[1:], columns = new_columns)
    pivot_df.drop(columns= pivot_df.columns[5:21], inplace= True)
    pivot_df.drop(index= [1256,1257], inplace= True)

    archivo= f'../data/raw/reservas_{sheet_name}_2023.parquet'
    pq.write_table(pa.Table.from_pandas(pivot_df), archivo)
        

['OPERADOR', 'CUENCA', 'PROVINCIA', 'CONCESIÓN O PERMISO', 'YACIMIENTO', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)']
29
['OPERADOR', 'CUENCA', 'PROVINCIA', 'CONCESIÓN O PERMISO', 'YACIMIENTO', '1(Mm3)', '2(MMm3)', '3(Mm3)', '4(MMm3)', '5(Mm3)', '6(MMm3)', '7(Mm3)', '8(MMm3)', '9(Mm3)', '10(MMm3)', '11(Mm3)', '12(MMm3)', '13(Mm3)', '14(MMm3)', '15(Mm3)', '16(MMm3)', 'reserv_comp_pet_(Mm3)', 'reserv_comp_gas_(MMm3)', 'reserv_prob_pet_(Mm3)', 'reserv_prob_gas_(MMm3)', 'reserv_pos_pet_(Mm3)', 'reserv_pos_gas_(MMm3)', 'recur_cont_pet_(Mm3)', 'recur_cont_gas_(MMm3)']
['1(Mm3)', '2(MMm3)', '3(Mm3)', '4(MMm3)', '5(Mm3)', '6(MMm3)', '7(Mm3)', '8(MMm3)', '9(Mm3)', '10(MMm3)', '11(Mm3)', '12(MMm3)', '13(Mm3)', '14(MMm3)', '15(Mm3)', '16(MMm3)']


In [28]:
reservas = pd.read_parquet('../data/raw/reservas_fin de vida util_2023.parquet')

In [29]:
reservas

,OPERADOR,CUENCA,PROVINCIA,CONCESIÓN O PERMISO,YACIMIENTO,reserv_comp_pet_(Mm3),reserv_comp_gas_(MMm3),reserv_prob_pet_(Mm3),reserv_prob_gas_(MMm3),reserv_pos_pet_(Mm3),reserv_pos_gas_(MMm3),recur_cont_pet_(Mm3),recur_cont_gas_(MMm3)
0,ALIANZA PETROLERA ARGENTINA S.A.,NEUQUINA,Mendoza,LOMA DE LA MINA,CERRO ALQUITRAN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ALIANZA PETROLERA ARGENTINA S.A.,NEUQUINA,Mendoza,LOMA DE LA MINA,LOMA DE LA MINA,73.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ALIANZA PETROLERA ARGENTINA S.A.,NEUQUINA,Mendoza,LOMA DE LA MINA,PUESTO LA QUEBRADA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CAPETROL ARGENTINA S.A.,GOLFO SAN JORGE,Chubut,RIO MAYO,RIO MAYO,45.0,27.0,12.0,7.0,11.0,7.0,0.0,0.0
4,CAPETROL ARGENTINA S.A.,GOLFO SAN JORGE,Chubut,SARMIENTO,CERRO GUACHO,1.0,1.0,1.0,1.0,1.0,1.0,0.0,101.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251,SELVA MARIA OIL S.A.,AUSTRAL,Santa Cruz,PALERMO AIKE,CAMPO LIMITE,0.0,0.0,0.0,0.0,0.0,0.0,4.8,83.6
1252,SELVA MARIA OIL S.A.,AUSTRAL,Santa Cruz,PALERMO AIKE,CERRO TRES HERMANOS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1253,SELVA MARIA OIL S.A.,AUSTRAL,Santa Cruz,PALERMO AIKE,MONTE AYMOND,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1254,SELVA MARIA OIL S.A.,AUSTRAL,Santa Cruz,PALERMO AIKE,PALERMO AIKE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
